# Analyzing Streaming Service Content in SQL

We will utilize streaming platform data to answer some questions related to expectations of a stakeholder.

## Exploring our data
Let's start by checking out the data we will be working with. We can start with the `amazon`, `hulu`, `netflix`, and `disney` tables.

In [ ]:
-- Explore the data in the table
SELECT *
FROM amazon
LIMIT 5

,id,title,year,age,imdb,rotten_tomatoes,type
0,4928,Fist of Fury,1972,18+,NaN,69/100,0
1,6813,Bazodee,2016,13+,NaN,47/100,0
2,4929,Fanboys,2009,13+,NaN,69/100,0
3,3784,Underworld: Evolution,2006,18+,NaN,76/100,0
4,3788,Across the Universe,2007,13+,NaN,76/100,0


In [ ]:
-- check the hulu table
SELECT *
FROM hulu
LIMIT 5

,id,title,year,age,imdb,rotten_tomatoes,type
0,3863,Bound,1996,18+,None,72/100,0
1,1971,Firefly,2002,16+,9.0/10,90/100,1
2,4091,Brian Banks,2019,13+,None,63/100,0
3,4083,Assassination Nation,2018,18+,None,64/100,0
4,4092,Soul Food,1997,18+,None,63/100,0


In [ ]:
-- check the netflix table
SELECT *
FROM netflix
LIMIT 5

,id,title,year,age,imdb,rotten_tomatoes,type
0,3606,Match,2017,NaN,NaN,32/100,0
1,2561,The Binding,2020,NaN,NaN,47/100,0
2,2215,ATM,2012,NaN,NaN,50/100,0
3,3220,Look for a Star,2009,NaN,NaN,40/100,0
4,3605,Arango y Sanint: Ríase el show,2018,NaN,NaN,32/100,0


In [ ]:
-- check Disney
SELECT *
FROM disney
LIMIT 5

,id,title,year,age,imdb,rotten_tomatoes,type
0,4132,The Kid,2019,18+,NaN,62/100,0
1,9198,Hatching Pete,2009,all,NaN,52/100,0
2,9200,Chef Donald,1941,None,NaN,52/100,0
3,9201,Tiger Cruise,2005,7+,NaN,52/100,0
4,9204,Amy,1981,all,NaN,52/100,0


We can also inspect the `genres` table, which is different from the other tables.

In [ ]:
-- select genre table
SELECT *
FROM genres
LIMIT 5

,film,genre
0,Sara's Notebook,"Dramas, International Movies, Thrillers"
1,Rare Exports: A Christmas Tale,"Action, Adventure, Comedy"
2,Gretel & Hansel,"Drama, Horror, Mystery"
3,Mr. Jones,"Drama, History, Thriller"
4,The Limehouse Golem,"Crime, Mystery, Thriller"


## Preparing our data
### Joining the different tables
Our data appears to mostly have the same column names. So we can join the data with a series of `UNION` which will append each table to the previous one.

We use `UNION ALL` to preserve any possible duplicate rows, as we will want to count entries if they appear in multiple services.

In [ ]:
-- make sure to add the service as a marker
-- we want to remember which platform the media comes from regardless if its one or multiple
SELECT *, 'amazon' as platform
FROM amazon
UNION ALL 
-- UNION ALL includes all records, UNION will drop duplicates
-- some media could be cross-shared across multiple platforms, it is important to append all records
SELECT * , 'hulu' as platform
FROM hulu
UNION ALL
SELECT *, 'netflix' as platform
FROM netflix
UNION ALL
SELECT * , 'disney' as platform
FROM disney

,id,title,year,age,imdb,rotten_tomatoes,type,platform
0,4928,Fist of Fury,1972,18+,None,69/100,0,amazon
1,6813,Bazodee,2016,13+,None,47/100,0,amazon
2,4929,Fanboys,2009,13+,None,69/100,0,amazon
3,3784,Underworld: Evolution,2006,18+,None,76/100,0,amazon
4,3788,Across the Universe,2007,13+,None,76/100,0,amazon
...,...,...,...,...,...,...,...,...
10159,5521,Sydney to the Max,2019,all,6.3/10,52/100,1,disney
10160,5518,Silver Surfer,1998,7+,7.1/10,52/100,1,disney
10161,5558,The Mickey Mouse Club,1955,all,7.7/10,48/100,1,disney
10162,2152,Genius,2017,16+,8.3/10,74/100,1,disney


### Inspecting missing data
It looks like we are missing some values in the `age` and `imdb` columns. We will also check the `rotten_tomatoes` column because we may use it later. Let's see how extensive this problem is.

To calculate the null values per column, we will use a combination of `SUM()` and `CASE WHEN` to count the number of null values.

In [ ]:
-- utilize another CTE to define the second part of the query
-- we want to look at null values as we notice IMDB tends to be empty
WITH platform_data as (
	SELECT *, 'amazon' as platform
	FROM amazon
	UNION ALL 
	SELECT * , 'hulu' as platform
	FROM hulu
	UNION ALL
	SELECT *, 'netflix' as platform
	FROM netflix
	UNION ALL
	SELECT * , 'disney' as platform
	FROM disney
),

all_media as(
	SELECT *
	FROM platform_data as pd
	LEFT JOIN genres as g 
		ON pd.title = g.film
)

-- count the total number of missing values with sum and case
-- expect a lot of missing values for IMDB column
-- investigate why
SELECT
	SUM(CASE WHEN title IS NULL THEN 1 ELSE 0 END) AS title_missing
	, SUM(CASE WHEN year IS NULL THEN 1 ELSE 0 END) AS year_missing
	, SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS age_missing
	, SUM(CASE WHEN imdb IS NULL THEN 1 ELSE 0 END) AS imdb_missing
	, SUM(CASE WHEN rotten_tomatoes IS NULL THEN 1 ELSE 0 END) AS rotten_missing
	, SUM(CASE WHEN type IS NULL THEN 1 ELSE 0 END) AS type_missing
	, SUM(CASE WHEN genre IS NULL THEN 1 ELSE 0 END) AS genre_missing
FROM all_media

,title_missing,year_missing,age_missing,imdb_missing,rotten_missing,type_missing,genre_missing
0,0,0,3633,6901,7,0,0


In [ ]:
WITH platform_data as (
	SELECT *, 'amazon' as platform
	FROM amazon
	UNION ALL 
	SELECT * , 'hulu' as platform
	FROM hulu
	UNION ALL
	SELECT *, 'netflix' as platform
	FROM netflix
	UNION ALL
	SELECT * , 'disney' as platform
	FROM disney
),

all_media as(
	SELECT *
	FROM platform_data as pd
	LEFT JOIN genres as g 
		ON pd.title = g.film
)

SELECT type
, COUNT(imdb)
FROM all_media
GROUP BY type
-- we can tell there lots of null values here, because COUNT(<var>) returns

,type,count
0,0,0
1,1,3263


## Analyzing our data
### Which is the most family-friendly streaming service?
Let's start by looking at the most family-friendly streaming service by the percentage of content geared towards children.

We have our primary genre column, but that could leave out some content. A better way may be to use pattern matching to find any references to "kids", "family", etc.

In [ ]:
WITH platform_data as (
	SELECT *, 'amazon' as platform
	FROM amazon
	UNION ALL 
	SELECT * , 'hulu' as platform
	FROM hulu
	UNION ALL
	SELECT *, 'netflix' as platform
	FROM netflix
	UNION ALL
	SELECT * , 'disney' as platform
	FROM disney
),

all_media as(
	SELECT *
	FROM platform_data as pd
	LEFT JOIN genres as g 
		ON pd.title = g.film
)

-- family friendly need to look at genre column and parse through
-- pattern match flags to identify children, kids and family tags
-- ILIKE for case insensitive
SELECT *
FROM all_media
WHERE genre ILIKE '%kids%'
	OR genre ILIKE '%family%'
	OR genre ILIKE '%children%'

,id,title,year,age,imdb,rotten_tomatoes,type,platform,film,genre
0,4961,Open Season,2006,7+,None,69/100,0,amazon,Open Season,"Children & Family Movies, Comedies"
1,5000,How to Steal a Dog,2014,None,None,68/100,0,amazon,How to Steal a Dog,"Drama, Kids"
2,4827,The Little Prince,2015,7+,None,74/100,0,amazon,The Little Prince,"Animation, Kids"
3,4864,Pinocchio,2019,13+,None,72/100,0,amazon,Pinocchio,"Animation, Comedy, Family, Fantasy, Musical"
4,5166,Troop Zero,2019,7+,None,64/100,0,amazon,Troop Zero,Kids
...,...,...,...,...,...,...,...,...,...,...
1536,5522,Vampirina,2017,all,6.6/10,52/100,1,disney,Vampirina,"Animation, Short, Comedy, Family, Fantasy, Mus..."
1537,5521,Sydney to the Max,2019,all,6.3/10,52/100,1,disney,Sydney to the Max,"Comedy, Family"
1538,5558,The Mickey Mouse Club,1955,all,7.7/10,48/100,1,disney,The Mickey Mouse Club,"Family, Comedy, Drama, Music"
1539,2152,Genius,2017,16+,8.3/10,74/100,1,disney,Genius,"Comedy, Family, Romance, Sci-Fi"


Great! That seems to be working. Let's adapt our query and use `CASE WHEN` to perform an aggregation and see which platform has the highest percentage of family content.

In [ ]:
WITH platform_data as (
	SELECT *, 'amazon' as platform
	FROM amazon
	UNION ALL 
	SELECT * , 'hulu' as platform
	FROM hulu
	UNION ALL
	SELECT *, 'netflix' as platform
	FROM netflix
	UNION ALL
	SELECT * , 'disney' as platform
	FROM disney
),

all_media as(
	SELECT *
	FROM platform_data as pd
	LEFT JOIN genres as g 
		ON pd.title = g.film
)

-- create flags for kids friendly content
-- we want to make sure we get the percent of media that is family friendly for each platform
SELECT platform, 
	AVG(CASE WHEN genre ILIKE '%kids%'
		OR genre ILIKE '%family%'
		OR genre ILIKE '%children%' THEN 1.0 ELSE 0.0 END) * 100 AS family_percent
FROM all_media
GROUP BY platform

,platform,family_percent
0,hulu,10.985075
1,netflix,11.057287
2,disney,74.744898
3,amazon,8.319242


### Which has the highest-rated content?
We also have access to information on the ratings of each piece of content in the `rotten_tomatoes` column. We use `SPLIT_PART()` to extract the number from the column and then cast (`::`) the result as a numeric.

We will also further split the data by movie and tv shows and visualize the result in a grouped bar chart.

In [ ]:
WITH platform_data as (
	SELECT *, 'amazon' as platform
	FROM amazon
	UNION ALL 
	SELECT * , 'hulu' as platform
	FROM hulu
	UNION ALL
	SELECT *, 'netflix' as platform
	FROM netflix
	UNION ALL
	SELECT * , 'disney' as platform
	FROM disney
),

all_media as(
	SELECT *
	FROM platform_data as pd
	LEFT JOIN genres as g 
		ON pd.title = g.film
)

-- need to split because the score has a / between the aggregate rating and 100
-- we know this column will be text as well
-- make sure to list 1 as its the first part of the split
SELECT platform
, CASE WHEN type = 1 THEN 'TV' ELSE 'Movie' END AS type
, AVG(SPLIT_PART(rotten_tomatoes, '/', 1) :: NUMERIC) as avg_rt_score
FROM all_media
GROUP BY platform, type
ORDER BY platform, type

,platform,type,avg_rt_score
0,amazon,Movie,51.990146
1,amazon,TV,52.377207
2,disney,Movie,60.047934
3,disney,TV,54.486034
4,hulu,Movie,60.482517
5,hulu,TV,59.690625
6,netflix,Movie,54.965913
7,netflix,TV,54.229586


### Have critics and audiences diverged over time?
Okay, for our final analysis, lets put the service aside and look into whether critics and audiences were more aligned on tv shows in the past.

To prepare the date for the chart cell, we will need to use `TO_DATE()` to convert the year into a date.

In [ ]:
WITH platform_data as (
	SELECT *, 'amazon' as platform
	FROM amazon
	UNION ALL 
	SELECT * , 'hulu' as platform
	FROM hulu
	UNION ALL
	SELECT *, 'netflix' as platform
	FROM netflix
	UNION ALL
	SELECT * , 'disney' as platform
	FROM disney
),

all_media as(
	SELECT *
	FROM platform_data as pd
	LEFT JOIN genres as g 
		ON pd.title = g.film
)


-- look at difference in audience (IMDB) to critics (Rotten Tomatoes)
-- TO_DATE will produce the start of the year as in jan 1 00:00
SELECT 
	date
	, AVG(ABS(rotten_score - imdb_score)) as avg_diff
FROM (
	SELECT TO_DATE(year::TEXT, 'YYYY') as date
	, SPLIT_PART(rotten_tomatoes, '/', 1) :: NUMERIC AS rotten_score
	, SPLIT_PART(imdb, '/', 1) :: NUMERIC * 10 AS imdb_score
	FROM all_media
	-- too many nulls in IMDB and RT, need to remove them
	WHERE imdb IS NOT NULL 
		AND rotten_tomatoes IS NOT NULL
		AND year >= 2000
	) as subquery
GROUP BY date
ORDER BY date

,date,avg_diff
0,2000-01-01 00:00:00+00:00,12.857143
1,2001-01-01 00:00:00+00:00,10.941176
2,2002-01-01 00:00:00+00:00,12.526316
3,2003-01-01 00:00:00+00:00,11.103448
4,2004-01-01 00:00:00+00:00,10.741935
5,2005-01-01 00:00:00+00:00,10.962264
6,2006-01-01 00:00:00+00:00,12.092593
7,2007-01-01 00:00:00+00:00,13.825000
8,2008-01-01 00:00:00+00:00,13.155172
9,2009-01-01 00:00:00+00:00,11.759036


What are the most divisive shows?

In [ ]:
WITH platform_data as (
	SELECT *, 'amazon' as platform
	FROM amazon
	UNION ALL 
	SELECT * , 'hulu' as platform
	FROM hulu
	UNION ALL
	SELECT *, 'netflix' as platform
	FROM netflix
	UNION ALL
	SELECT * , 'disney' as platform
	FROM disney
),

all_media as(
	SELECT *
	FROM platform_data as pd
	LEFT JOIN genres as g 
		ON pd.title = g.film
)

-- repeat the same steps as before, but this time we need to include a grouping by show title
-- we will not exclude shows based on year, since we are already filtering by type of media
SELECT title
, SUM(ABS(rt_score - imdb_score)) as show_diff
FROM (
	SELECT title
	, SPLIT_PART(rotten_tomatoes, '/', 1) ::NUMERIC as rt_score
	, SPLIT_PART(imdb, '/', 1) :: NUMERIC * 10 as imdb_score
	FROM all_media
	WHERE type = 1 
		AND imdb IS NOT NULL
		AND rotten_tomatoes IS NOT NULL
	) as subquery
GROUP BY title
ORDER BY show_diff DESC

,title,show_diff
0,Molang,111.0
1,Pick of the Litter,102.0
2,On the Real,102.0
3,Special Forces,94.0
4,Destiny,90.0
...,...,...
2929,Hell's Kitchen,0.0
2930,Easy,0.0
2931,Adventure Time,0.0
2932,Archer,0.0
